# Data:
Data regarding existing, competitor restaurants within a 5 kilometer radius of the client’s home were obtained through a Foursquare API query, and entered into a pandas database. This data was then refined to narrow the database to feature only sushi restaurants, leaving a database of 19 competitors in the client’s target area. 

In addition to the type of restaurant, the API call also provided each competitor’s latitude and longitude coordinates, which could then be used to plot their location on a map.

# Methodology: 
After plotting the locations of the competitor sushi restaurants, as well as the client’s home, a visualization was created to approximate the service areas of the client’s competitors. Circles with two-kilometer radii were laid over the competitor locations, representing the area where competition was likely to be more intense given the proximity to a competitor sushi restaurant.


In [2]:
# Install and import appropriate packages:

!conda install -c conda-forge scikit-learn --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

import numpy as np
import pandas as pd
import json
import lxml

from IPython.display import Image 
from IPython.core.display import HTML 

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans


import folium

print("Done.")

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Done.


# DATA

In [3]:
# Our target location is Houston, so we will obtain data regarding our client's
# competitors with a Foursquare API Query.

# Set up Foursquare access:

CLIENT_ID = 'PVKFVTWRLLOZO0ADVKZZOEOFJUYFKELV0XADDQVALQTC0AJ0'
CLIENT_SECRET = 'PHTMQF32E1WH5DZXDKWKPFGCGC5JP3E1FEBRYCG323TQR5M4'
VERSION = '20180604'
LIMIT = 30

# The client does not want to commute more than 5000 meters from their place of residence, so we will begin with this address:
address = '5701 Main St, Houston, TX'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# Format Foursquare API query
QUERY = 'sushi'
RADIUS = 5000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, QUERY, RADIUS, LIMIT)

# Get query results

results = requests.get(url).json()

# Then convert results into a pandas dataframe

sushi_venues = results['response']['venues']
sushi_venues = pd.DataFrame(sushi_venues)
sushi_venues.shape

(30, 8)

In [4]:
# Data Processing - Let's see if we can filter out irrelevant information:

# Keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories','location','id']
sushi_venues_filtered = sushi_venues.loc[:, filtered_columns]

# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Filter the category for each row
sushi_venues_filtered['categories'] = sushi_venues_filtered.apply(get_category_type, axis=1)

# Clean column names by keeping only last term
sushi_venues_filtered.columns = [column.split('.')[-1] for column in sushi_venues_filtered.columns]

# Now let's retain only the restaurants that are classified as "Sushi Restaurant":

sushi_venues_filtered = sushi_venues_filtered[(sushi_venues_filtered['categories']=='Sushi Restaurant')]
sushi_venues_filtered.reset_index(drop=True,inplace=True)

# Now we're down to 19 restaurants!

sushi_venues_filtered

,name,categories,location,id
0,MF Sushi,Sushi Restaurant,"{'address': '1401 Binz St Ste 100', 'lat': 29....",55a406dc498e4cf048794137
1,The Fish Restaurant & Sushi Bar,Sushi Restaurant,"{'address': '309 Gray St #107', 'crossStreet':...",4ad75ab2f964a520c40921e3
2,Oishii Japanese Restaurant & Sushi Bar,Sushi Restaurant,"{'address': '3764 Richmond Ave', 'crossStreet'...",4bb650d02ea19521197bab2f
3,akashi Sushi (Akashi Sushi),Sushi Restaurant,"{'lat': 29.69136994570386, 'lng': -95.41314186...",58751d6c3bd4ab73fed2df78
4,RA Sushi,Sushi Restaurant,"{'address': '3908 Westheimer Rd', 'crossStreet...",4ad7a9f9f964a520760d21e3
5,Kura Revolving Sushi Bar,Sushi Restaurant,"{'address': '3510 A-Main Street', 'crossStreet...",5ab58d00a4b51b3dc0e99260
6,Sushi King,Sushi Restaurant,"{'address': '3401 Kirby Dr', 'crossStreet': 'R...",4b7624dcf964a520be3f2ee3
7,Aka Sushi House,Sushi Restaurant,"{'address': '2390 W Alabama St', 'lat': 29.738...",4afb5f4cf964a520701d22e3
8,Rokaru Sushi,Sushi Restaurant,"{'address': '3600 Main St', 'lat': 29.73907999...",560ae61e498e8ce7dc85918c
9,Fukuoka Sushi Bar & Grill,Sushi Restaurant,"{'address': '2808 Milam St', 'crossStreet': 'D...",5bbcdb86e96d0c002c4ce78c


# METHODOLOGY

In [5]:
# Now let's see our competitors on a map:

sushi_venues_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add a red circle marker to represent our client's home

folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='green',
    popup='Client Home',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(sushi_venues_map)

# Add the sushi restaurants as blue circle markers
for row in sushi_venues_filtered['location']:
    lat = row['lat']
    lng = row['lng']
    label = "Sushi Restaurant"
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sushi_venues_map)

sushi_venues_map

In [6]:
# Now we need to find some areas in the map that are not near any competitor sushi restaurants while still being close
# to the client's home. To do this, we will replace the CircleMaker points with Circle points; this way, we can create circles
# of a fixed radius length. The client has decided that they do not want their restaurant to be within 2 kilometers of any
# competitor.

for row in sushi_venues_filtered['location']:
    lat = row['lat']
    lng = row['lng']
    label = None
    folium.Circle(
        [lat, lng],
        radius=2000,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.3
    ).add_to(sushi_venues_map)

sushi_venues_map

# DISCUSSION OF RESULTS

Now that the client has this map, they can begin exploring sites that are not within the red circles.

Possible improvements:
- Future iterations of this model might benefit from analzing population density and consumer habits
- A more sophisticated model could find the point on that is furthest away from the competitors while still being near the client's home

Thank you for considering my report.

